#  Part of Speech

Autora: Rita Rezende Borges de Lima

O objetivo principal do trabalho consiste de treinar e validar um modelo de  *Part of Speech* (POS) para a lingua portuguesa, utilizando como corpus o conjunto de dados **Mac-Morpho**.

In [1]:
import pandas as pd
import numpy as np
import spacy, re, nltk
import matplotlib as mpl
import seaborn as sns

from nltk.tag import hmm
from nltk.tokenize import word_tokenize, sent_tokenize
from sklearn.metrics import classification_report


/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.8) or chardet (4.0.0) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


## Preprocessamento de Dados

A **primeira etapa** realizada foi a preparação dos dados utilizados. Foi realizada a leitura dos arquivos de corpus *macmorpho-train.txt* e *macmorpho-test.txt*. A base de treinamento contém 36843 sentenças tageadas em portugês, já a base de teste contém 9513 sentenças. Ou seja, a base de dados está dividida em treinamento e testes com uma proporção 80% e 20% dos dados, respectivamente.

In [2]:
def word_tag_separation(train_tokens):
    labelled_sentence = []
    sentence = ''
    for element in train_tokens:
        if '_' in element:
            word, tag = element.split('_')
            labelled_sentence.append((word.lower(), tag))
            sentence += word + ' '
    return labelled_sentence, sentence

In [3]:
def sentence_tokenization(set):
    sentences, cur_sentence = [], []

    for token in set:
        cur_sentence.append(token)
        if token[0] == '.' or token[0] == ':' or token[0] == ';' or  token[0] == '!' or token[0] == '?' or token[0] == '…':
            sentences.append(cur_sentence)
            cur_sentence = []
            
    return sentences

O texto foi transformado em uma lista de frases que por sua vez é uma lista de palavras. Estas palavras consistem de tuplas onde o primeiro valor é a palavra toda em caixa baixa e o segundo é sua etiqueta.

In [4]:
train_text = open('macmorpho-train.txt', encoding='utf-8').read()
train_tokens = re.sub('\n', ' ', train_text).split(' ')

train_set, train_text = word_tag_separation(train_tokens)
train_sentences = sentence_tokenization(train_set)

print("Quantidade de palavras do conjunto de treinamento: ", len(train_set))
print("Quantidade de sentencas do conjunto de treinamento: ", len(train_sentences))

train_sentences[0]

Quantidade de palavras do conjunto de treinamento:  728497
Quantidade de sentencas do conjunto de treinamento:  36843


[('jersei', 'N'),
 ('atinge', 'V'),
 ('média', 'N'),
 ('de', 'PREP'),
 ('cr$', 'CUR'),
 ('1,4', 'NUM'),
 ('milhão', 'N'),
 ('na', 'PREP+ART'),
 ('venda', 'N'),
 ('da', 'PREP+ART'),
 ('pinhal', 'NPROP'),
 ('em', 'PREP'),
 ('são', 'NPROP'),
 ('paulo', 'NPROP'),
 ('.', 'PU')]

In [5]:
test_text = open('macmorpho-test.txt', encoding='utf-8').read()
test_tokens = re.sub('\n', ' ', test_text).split(' ')

test_set, test_text = word_tag_separation(test_tokens)
test_sentences = sentence_tokenization(test_set)

print("Quantidade de palavras do conjunto de teste: ", len(test_set))
print("Quantidade de sentencas do conjunto de teste: ", len(test_sentences))

test_sentences[1]

Quantidade de palavras do conjunto de teste:  178373
Quantidade de sentencas do conjunto de teste:  9513


[('número', 'N'),
 ('duplo', 'ADJ'),
 ('especial', 'ADJ'),
 (',', 'PU'),
 ('é', 'V'),
 ('inteirinho', 'ADJ'),
 ('dedicado', 'PCP'),
 ('a', 'PREP'),
 ('ensaios', 'N'),
 ('sobre', 'PREP'),
 ('moda', 'N'),
 ('.', 'PU')]

In [6]:
validation_text = open('macmorpho-dev.txt', encoding='utf-8').read()
validation_tokens = re.sub('\n', ' ', validation_text).split(' ')

validation_set, validation_text = word_tag_separation(validation_tokens)
validation_sentences = sentence_tokenization(validation_set)

print("Quantidade de palavras do conjunto de validação: ", len(validation_set))
print("Quantidade de sentencas do conjunto de validação: ", len(validation_sentences))

validation_sentences[1]

Quantidade de palavras do conjunto de validação:  38881
Quantidade de sentencas do conjunto de validação:  1956


[('porém', 'KC'),
 (',', 'PU'),
 ('como', 'KS'),
 ('a', 'ART'),
 ('previsão', 'N'),
 ('indica', 'V'),
 ('entrada', 'N'),
 ('de', 'PREP'),
 ('frente', 'N'),
 ('fria', 'ADJ'),
 (',', 'PU'),
 ('deve', 'V'),
 ('-se', 'PROPESS'),
 ('aguardar', 'V'),
 ('até', 'PREP'),
 ('quarta-feira', 'N'),
 ('e', 'KC'),
 ('irrigar', 'V'),
 ('somente', 'PDEN'),
 ('se', 'KS'),
 ('o', 'ART'),
 ('total', 'N'),
 ('de', 'PREP'),
 ('chuva', 'N'),
 ('for', 'V'),
 ('inferior', 'ADJ'),
 ('a', 'PREP'),
 ('30', 'NUM'),
 ('mm', 'N'),
 ('.', 'PU')]

## Treinamento dos Modelos

Após a preparação dos dados, iniciaremos o **treinamento do modelo**. Existem múltiplos modos de realizar o treinamento de um modelo de POS, neste trabalho focaremos em dois: um utilizando **Long Short Term Memory:** e **Hidden Markov Model:**.  

Estes foram utilizados, testados e validadados nesse trabalho. A seguir, são apresentados os processos de treinamento e validação.



### Treinamento Hidden Markov Model

In [7]:
trainer = hmm.HiddenMarkovModelTrainer()
model_hmm = trainer.train_supervised(train_sentences)

### Teste Hidden Markov Model
Como é possível ver abaixo o modelo HMM obteve um valor de acurácia de 67% na base de teste.

In [8]:
print("Accuracy HMM model: {}".format(model_hmm.evaluate(test_sentences) * 100))

Accuracy HMM model: 67.92171460927382


## Validação dos Modelos

### Funções utilizadas em ambas as validações

In [9]:
def generate_df_class_by_class(model, test):


    tagged_test_sentences = model.tag_sents([[token for token,tag in sent] for sent in test])
    y_test = [str(tag) for sentence in test for token,tag in sentence]
    y_pred = [str(tag) for sentence in tagged_test_sentences for token,tag in sentence]
    
    
    df = pd.DataFrame([[0] * 8] *8, columns, columns)
  
    for test, pred in zip(y_test, y_pred):
        if test in columns and pred in columns:
            df[test][pred] += 1
            
            
    return df

In [14]:
def generate_df_evaluate_by_class(model, test):

    tagged_test_sentences = model.tag_sents([[token for token,tag in sent] for sent in test])
    gold = [str(tag) for sentence in test for token,tag in sentence]
    pred = [str(tag) for sentence in tagged_test_sentences for token,tag in sentence]

    report = classification_report(gold, pred, output_dict=True)
    df = pd.DataFrame(report).transpose().reset_index()

    return df

In [11]:
def prettify_dataframes(df_hmm):
    def highlight_bg_correct_pred(s):
        def get_color(value):
            r = 235 - value/255
            g = 240 
            b = 230 - value/255
            return 'rgb({r}, {g}, {b})'.format(r=r, g=g, b=b)

        return ['background-color: {color}'.format(color = get_color(s[idx])) if idx == s._name else '' for idx in s.index]

    def highlight_bg_wrong_pred(s):
        def get_color(value):
            r = 235
            g = 160 - value/2
            b = 160 - value/2
            return 'rgb({r}, {g}, {b})'.format(r=r, g=g, b=b)

        return ['background-color: {color}'.format(color = get_color(s[idx])) if idx != s._name else '' for idx in s.index]

    return df_hmm.style.apply(highlight_bg_correct_pred, axis = 1)\
                        .apply(highlight_bg_wrong_pred, axis = 1)

### Validação Hidden Markov Model

Inicialmente para cada palavra em nosso conjunto de validação iremos tentar prever sua classe lexical e adicionar esta ocorrência em um dataframe para a respectiva tag correta a posição da tag encontrada. Ou seja, todos as ocorrências na diagonal do df (pintado de verde) representam ocorrências corretas, o restante (pintado de vermelho) representa ocorrências onde nosso modelo errou. A análise foi realizada para as oito classes mais comuns e esta pode ser vista a seguir:

In [12]:
columns = ["N", "NUM", "ART", "ADJ", "ADV", "NPROP", "PREP", "V"]
df_hmm = generate_df_class_by_class(model_hmm, validation_sentences)

prettify_dataframes(df_hmm)

,N,NUM,ART,ADJ,ADV,NPROP,PREP,V
N,8061,246,578,639,241,1502,1018,900
NUM,25,418,0,0,0,4,0,0
ART,2,28,2292,0,1,15,30,0
ADJ,73,0,0,1161,5,67,3,0
ADV,3,0,0,5,628,1,39,0
NPROP,117,2,1,25,0,2066,19,2
PREP,6,0,23,6,36,71,2636,1
V,14,0,0,2,1,7,1,3070


Por meio da biblioteca *scikit-learn* também geraremos um report com métricas como precisão, f1-score e suporte. A análise foi realizada para as oito classes mais comuns e esta pode ser vista a seguir:

In [15]:
df_hmm_results = generate_df_evaluate_by_class(model_hmm, validation_sentences)
df_hmm_results.loc[(df_hmm_results['index'].isin(columns))]

,index,precision,recall,f1-score,support
0,ADJ,0.879545,0.630293,0.734345,1842.0
1,ADV,0.872222,0.654849,0.748064,959.0
3,ART,0.961813,0.790617,0.867853,2899.0
8,N,0.472952,0.969569,0.635776,8314.0
9,NPROP,0.916593,0.543827,0.682637,3799.0
10,NUM,0.935123,0.600575,0.731409,696.0
13,PREP,0.936412,0.697908,0.799757,3777.0
25,V,0.990962,0.772327,0.868090,3975.0


## Conclusão

Para o modelo **Hidden Markov Model** foi possível observar uma maior precisão para as classes de verbo, preposição, numeral e nomes próprios. É também interessante ressaltar a baixa precisão para nomes, múltiplas palavras que não tinham essa classe foram classificadas assim erroneamente. Também é interessante ressaltar a alta quantidade de falsos positivos para nomes e nomes próprios que foram classificados de maneira errada como adjetivos e preposições. Artigos e Preposições também foram confundidos de ambos os modos. 